In [1]:
import time
from pylab import *
import numpy.linalg as LA
import numpy as np

def timeit(f):

    def timed(*args, **kw):

        ts = time.time()
        result = f(*args, **kw)
        te = time.time()

        print('func:%r took: %2.4f sec' % (f.__name__,  te-ts))
        return result

    return timed

# CHEM 277B: Homework 1 - Local Optimization Methods #

## 1. Bisection vs. Golden Section ##

<img src="Pictures/Bisection%20Graph.png" width="300" height="300">

In class we used the simple bisection method to take the first step in isolating at least one minimum for the function shown. This first step in placement of d reduced the original interval [a,b,c] =1.0 to [a,b,d] = 0.75. But in general, the average size interval <L> after Step 1 is determined by the equal probability of placing point d in either sub-interval, such that <L1>=P(left-interval) x 1⁄2 + P(right-interval) x 3⁄4 = 0.625 (since you can’t a priori know the best half)

#### (a) For step 2, place point e at the bisector of larger interval [a,b]. Why is this better than [b,d]? #####

<img src="Pictures/Bisection_e.png" width="300" height="300">

Placing point *e* at the bisector of larger interval $[a,b]$ is better than placing it at the bisector of $[b,d]$ because there is a greater chance of further reducing the interval. If point *e* were placed in $[b,d]$, while it could reduce the interval to $[b,d] = 0.25$ if $f(e) < f(b)$, if $f(e) > f(b)$, the interval would only be reduced to $[a,e] = 0.625$. On the other hand, point *e* placed at the bisector of $[b,d]$ would reduce the interval to $[a,b]$ or $[e,d]$, both of distance 0.5. This provides a higher chance of consistently reducing the search space. Placing points at the bisector of the larger interval could also prevent being trapped into a local minimum, as seen when point *e* is placed in $[b,d]$.

#### (b) What is the new interval and how much is the search space reduced? ####

The new interval according to the figure in problem 1a is $[a,e,b] = 0.5$, so the search spaced was reduced by $\frac{1}{3}$.

#### (c) For step 3, reduce the size of the interval from step 2 by placing point f at the bisection of your choice. ####

<img src="Pictures/1c.jpg" width="300" height="300">

Placing point *f* at the bisector of interval $[a,e]$ reduces the interval to $[f,e,b] = 0.375$, search spaced was reduced by $\frac{1}{4}$.

#### (d) fill in tree for all possible size intervals for steps 2 and 3. Write your answers in ratios to the interval size of the previous step. #### 

<img src="Pictures/1d.jpg" width="500" height="500">

 step 1: $\frac{1}{2}$ and $\frac{3}{4}$
 
 step 2: [$\frac{1}{2}$ : $\frac{3}{4}$ , $\frac{1}{2}$] and [$\frac{3}{4}$: $\frac{2}{3}$, $\frac{2}{3}$]
 
 step 3: [$\frac{3}{4}$: $\frac{2}{3}$, $\frac{2}{3}$],  [$\frac{1}{2}$: $\frac{1}{2}$, $\frac{3}{4}$], [$\frac{2}{3}$: $\frac{3}{4}$, $\frac{1}{2}$], [$\frac{2}{3}$: $\frac{3}{4}$, $\frac{1}{2}$]

#### (e) What is average size of interval at steps 2 and 3? ####

step 2: $P(0.25) \times \frac{3}{4} \times \frac{1}{2}+ P(0.25) \times \frac{1}{2} \times \frac{1}{2} + P(0.25) \times \frac{2}{3} \times \frac{3}{4} + P(0.25) \times \frac{2}{3} \times \frac{3}{4} = 0.406$

step 3: $P(0.125) \times \frac{2}{3} \times \frac{3}{4} \times \frac{1}{2} + P(0.125) \times \frac{2}{3} \times \frac{3}{4} \times \frac{1}{2} + P(0.125) \times \frac{1}{2} \times \frac{1}{2} \times \frac{1}{2} + P(0.125) \times \frac{3}{4} \times \frac{1}{2} \times \frac{1}{2} + P(0.125) \times \frac{3}{4} \times \frac{2}{3} \times \frac{3}{4}+ P(0.125) \times \frac{1}{2} \times \frac{2}{3} \times \frac{3}{4} + P(0.125) \times \frac{3}{4} \times \frac{2}{3} \times \frac{3}{4} + P(0.125) \times \frac{1}{2} \times \frac{2}{3} \times \frac{3}{4} = 0.258$

#### (f) How much does Golden Section improve over Bisection at each step? Please use a chart of steps v.s. different methods to show their difference. ####





Golden Section: 0.618, 0.382, 0.236
Bisection: 0.625, 0.646, 0.635

| Step | Bisection | Golden Section| 
|------|-----------|---------------|
|1  | 0.625 | 0.618 | 
|2 | 0.406 | 0.382 | 
| 3 | 0.258 | 0.236 | 

## 2. Local optimization using 1st and quasi-2nd order methods. ##

You will solve the following optimization problem using a python code you develop for the steepest descents method! For the function
𝑓(𝑥, 𝑦) = 𝑥4 − 𝑥2 + 𝑦2 + 2𝑥𝑦 − 2
there are three stationary points found over the range x = [- 2,2] and y = [-2,2].

#### (a) Starting from point (1.5,1.5), and with stepsize=0.1, determine new (𝑥, 𝑦) position using one step of the steepest descent algorithm (check against the debugging output). Is it a good optimization step? Depending on this outcome, how will you change the stepsize in the next step? #### 

In [2]:
def f(position):
    
    x, y = position[0], position[1]
    
    # return the evaluation of the function 
    return x**4 - x**2 + y**2 + 2*x*y - 2

def f_prime(position):
    
    x, y = position[0], position[1]
    
    # return the partial derivates of the function
    return np.array([4*x**3 - 2*x + 2*y, 2*y + 2*x])

def one_step_steepest_descent(func,first_derivative,starting_point,stepsize):
    # determine new (x,y) position using one step of steepest descent 
    
    # position after one step 
    new_position = starting_point - (stepsize*first_derivative(starting_point))
    
    print(f"new (x,y) position: {new_position}")
    print(f"f(x,y) for starting point: {f(starting_point)}")
    print(f"f(x,y) for new position: {f(new_position)}")

In [3]:
one_step_steepest_descent(f, f_prime, np.array([1.5, 1.5]), 0.1)

new (x,y) position: [0.15 0.9 ]
f(x,y) for starting point: 7.5625
f(x,y) for new position: -0.9419937500000004


This is a good step because $f(x_1, y_1) < f(x_0, y_0)$ meaning that we are moving along a negative gradient and thus closer to the minimum. The stepsize will be multiplied by 1.2 since it was a good step.

#### (b) Implement the steepest decent using the provided template. Continue executing steepest descents. How many steps does it take to converge to the local minimum to tolerance = 1 x 10-5 of the gradient (check against the debugging output and compare code timings)?
Note: You don’t need to use line search, just take one step in the search direction, and use the following stepsize update: 𝜆 = {1.2 𝜆 𝑓𝑜𝑟 𝑎 𝑔𝑜𝑜𝑑 𝑠𝑡𝑒𝑝, 0.5𝜆 𝑓𝑜𝑟 𝑎 𝑏𝑎𝑑 𝑠𝑡𝑒𝑝}

In [4]:
@timeit
def steepest_descent(func,first_derivative,starting_point,stepsize,tol):
    # evaluate the gradient at starting point
    
    count = 0
    visited = [starting_point]
    deriv = first_derivative(starting_point)

    while LA.norm(deriv) > tol and count < 1e6:
        # calculate new point position
        new_position = visited[-1] - (stepsize*first_derivative(visited[-1]))
        deriv = first_derivative(new_position)
                                         
        if func(new_position) < func(visited[-1]):
            # the step makes function evaluation lower - it is a good step. what do you do?
            stepsize *= 1.2 
            
        else:
            # the step makes function evaluation higher - it is a bad step. what do you do?
            stepsize *= 0.5 
        
        visited.append(new_position)
        count += 1
                                         
    # return the results
    return {"minimum":new_position,"evaluation":func(new_position), 
            "steps":count, "path":np.asarray(visited)}

In [5]:
steepest_descent(f, f_prime, np.array([1.5, 1.5]), 0.1, 1e-5)

func:'steepest_descent' took: 0.0025 sec


{'minimum': array([-0.99999892,  0.99999943]),
 'evaluation': -2.999999999995068,
 'steps': 45,
 'path': array([[ 1.5       ,  1.5       ],
        [ 0.15      ,  0.9       ],
        [-0.03162   ,  0.648     ],
        [-0.22733235,  0.47048256],
        [-0.4603766 ,  0.38644985],
        [-0.73063964,  0.41710875],
        [-0.91361447,  0.57314178],
        [-0.89067253,  0.77647099],
        [-1.072703  ,  0.85831194],
        [-0.88004038,  0.93513214],
        [-1.07440969,  0.91144369],
        [-0.8191814 ,  0.99553056],
        [-1.00371455,  0.95003442],
        [-0.98247032,  0.96665308],
        [-1.00196259,  0.97252925],
        [-0.98533102,  0.98565078],
        [-1.0162044 ,  0.98547972],
        [-0.99022477,  0.99369805],
        [-1.00370679,  0.9925832 ],
        [-0.9936794 ,  0.99686773],
        [-1.00672832,  0.99539405],
        [-0.99782619,  0.99801346],
        [-1.00028169,  0.99796153],
        [-0.99913443,  0.99873366],
        [-1.00035525,  0.9988937

It took 45 steps to converge to the local minimum (-1.0, 1.0).

#### (c) Compare your steepest descent code against conjugate gradients (CG), and BFGS to determine the local minimum starting from (1.5,1.5). In terms of number of steps, are conjugate gradients and/or BFGS more efficient than steepest descents? ####

In [6]:
from scipy.optimize import minimize

minimize(f, np.array([1.5, 1.5]), method='CG', options={'disp': True, 'gtol': 1e-5})

Optimization terminated successfully.
         Current function value: -3.000000
         Iterations: 9
         Function evaluations: 78
         Gradient evaluations: 26


     fun: -2.99999999999959
     jac: array([ 2.08616257e-07, -1.10268593e-06])
 message: 'Optimization terminated successfully.'
    nfev: 78
     nit: 9
    njev: 26
  status: 0
 success: True
       x: array([-0.99999984,  0.99999929])

In [7]:
minimize(f, np.array([1.5, 1.5]), method='BFGS', options={'disp': True, 'gtol': 1e-5})

Optimization terminated successfully.
         Current function value: -3.000000
         Iterations: 7
         Function evaluations: 24
         Gradient evaluations: 8


      fun: -2.9999999999998255
 hess_inv: array([[ 0.12457729, -0.12457659],
       [-0.12457659,  0.62569812]])
      jac: array([-1.63912773e-06, -2.98023224e-08])
  message: 'Optimization terminated successfully.'
     nfev: 24
      nit: 7
     njev: 8
   status: 0
  success: True
        x: array([ 0.99999979, -0.9999998 ])

Conjugate gradients took 9 steps while BFGS took 7 steps. The steepest descent code took 45 steps, so CG and BFGS are much more efficient.

## 3. Local optimization and machine learning using Stochastic Gradient Descent (SGD). ##

The Rosenbrock Banana Function looks innocuous enough
$𝑓(𝑥,𝑦)=(1−𝑥)^2 +10(𝑦−𝑥^2)^2$ with only one (global) minimum at $(𝑥, 𝑦) = (1.0,1.0)$!

#### (a) Starting at 𝑥 = −0.5 and 𝑦 = 1.5, and using your code for steepest descents with stepsize =0.1, how many steps to converge to the minimum? Use a tolerance = 1 x 10-5 #### 

In [8]:
def rosenbrock(x):
    return ((1 - x[0])**2 + 10*(x[1] - x[0]**2)**2)

def derive_rosenbrock(x):
    return np.array([-2*(1 - x[0]) - 40*x[0]*(x[1] - x[0]**2), 20*(x[1] - x[0]**2)])

In [9]:
steepest_descent(rosenbrock, derive_rosenbrock, np.array([-0.5, 1.5]), 0.1, 1e-5)

func:'steepest_descent' took: 0.0020 sec


/var/folders/xf/ys07020s0656mhx3qrl86vfh0000gn/T/ipykernel_11894/1161317273.py:5: RuntimeWarning: overflow encountered in double_scalars
  return np.array([-2*(1 - x[0]) - 40*x[0]*(x[1] - x[0]**2), 20*(x[1] - x[0]**2)])
/var/folders/xf/ys07020s0656mhx3qrl86vfh0000gn/T/ipykernel_11894/1161317273.py:2: RuntimeWarning: overflow encountered in double_scalars
  return ((1 - x[0])**2 + 10*(x[1] - x[0]**2)**2)
/var/folders/xf/ys07020s0656mhx3qrl86vfh0000gn/T/ipykernel_11894/468941871.py:11: RuntimeWarning: invalid value encountered in subtract
  new_position = visited[-1] - (stepsize*first_derivative(visited[-1]))


{'minimum': array([nan, inf]),
 'evaluation': nan,
 'steps': 8,
 'path': array([[-5.00000000e-001,  1.50000000e+000],
        [-2.70000000e+000, -1.00000000e+000],
        [ 4.24360000e+001,  7.29000000e+000],
        [-7.60696243e+004,  9.04052048e+002],
        [ 2.20091744e+014,  1.44664761e+009],
        [-2.66533168e+042,  6.05504695e+027],
        [ 2.36681219e+126,  4.43999561e+083],
        [            -inf,  1.75056248e+251],
        [             nan,              inf]])}

In [10]:
steepest_descent(rosenbrock, derive_rosenbrock, np.array([-0.5, 1.5]), 0.01, 1e-5)

func:'steepest_descent' took: 0.0359 sec


{'minimum': array([0.99999095, 0.9999816 ]),
 'evaluation': 8.280022388342527e-11,
 'steps': 1165,
 'path': array([[-0.5       ,  1.5       ],
        [-0.72      ,  1.25      ],
        [-0.93156096,  1.074416  ],
        ...,
        [ 0.99999058,  0.99998155],
        [ 0.99999099,  0.99998145],
        [ 0.99999095,  0.9999816 ]])}

Starting from x = -0.5 and y = 1.5 and using a stepsize of 0.1, the steepest descent function is unable to converge to the minimum, and overflow occurs really fast. Infinite values is reached after 8 steps. However, when I changed the stepsize to 0.01, it was able to find the (1.0, 1.0) minimum after 1165 steps. 

#### (b) By adding a small amount of stochastic noise to the gradient at every step (In your code add a random vector that is the same norm as the gradient at that step), which is equivalent to a small batch derivative of any loss function in deep learning, implement your own stochastic gradient descent code by modifying on your steepest descent code, and run the SGD algorithm. (Check against debugging outputs.) ####

In [11]:
@timeit
def stochastic_gradient_descent(func,first_derivative,starting_point,
                                stepsize,tol=1e-5,stochastic_injection=1):
    '''stochastic_injection: controls the magnitude of stochasticity (multiplied with stochastic_deriv)
        0 for no stochasticity, equivalent to SD. 
        Use 1 in this homework to run SGD
    '''
    # evaluate the gradient at starting point
    
    count=0
    visited=[starting_point]
    deriv = first_derivative(starting_point)
    
    while LA.norm(deriv) > tol and count < 1e5:
        if stochastic_injection>0:
            # formulate a stochastic_deriv that is the same norm as your gradient 
            # random vector (mean:0, sd:1, size:2)
            stochastic_deriv = np.random.normal(0,1,len(starting_point))  
            # calculate same norm as deriv 
            stochastic_deriv = stochastic_deriv/LA.norm(stochastic_deriv)*LA.norm(deriv) 
        else:
            stochastic_deriv=np.zeros(len(starting_point))
        
        # calculate new point position
        direction =- (deriv + stochastic_injection*stochastic_deriv)
        new_position = visited[-1] + stepsize*direction
        deriv = first_derivative(new_position)
        
        if func(new_position) < func(visited[-1]):
            # the step makes function evaluation lower - it is a good step. what do you do?
            stepsize *= 1.2
            
        else:
            # the step makes function evaluation higher - it is a bad step. what do you do?
            stepsize *= 0.5
            
        visited.append(new_position)
        count+=1
        
    return {"minimum":new_position,"evaluation":func(new_position), 
            "steps":count, "path":np.asarray(visited)}

In [12]:
stochastic_gradient_descent(rosenbrock, derive_rosenbrock, np.array([-0.5, 1.5]), 0.01)

func:'stochastic_gradient_descent' took: 0.0853 sec


{'minimum': array([0.99999152, 0.9999826 ]),
 'evaluation': 7.384049766997677e-11,
 'steps': 2023,
 'path': array([[-0.5       ,  1.5       ],
        [-0.59010287,  0.94336221],
        [-0.58830064,  0.65936119],
        ...,
        [ 0.99999136,  0.99998255],
        [ 0.99999143,  0.99998267],
        [ 0.99999152,  0.9999826 ]])}

#### (c) evaluate how much better or worse is the SGD convergence against the CG or BFGS method to find the global minimum, in terms of number of steps. Converge function/gradient to tolerance =1 × 10-5 ####

CG:

In [13]:
minimize(rosenbrock, np.array([-0.5, 1.5]), method='CG', 
         options={'disp': True, 'gtol': 1e-5})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 132
         Gradient evaluations: 44


     fun: 2.0711734599818967e-13
     jac: array([ 4.95077415e-08, -2.45421994e-08])
 message: 'Optimization terminated successfully.'
    nfev: 132
     nit: 20
    njev: 44
  status: 0
 success: True
       x: array([0.99999955, 0.99999908])

BFGS:

In [14]:
minimize(rosenbrock, np.array([-0.5, 1.5]), method='BFGS', 
         options={'disp': True, 'gtol': 1e-5})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 22
         Function evaluations: 93
         Gradient evaluations: 31


      fun: 1.6856836004019217e-13
 hess_inv: array([[0.50988602, 1.01962714],
       [1.01962714, 2.08896666]])
      jac: array([ 1.15312325e-07, -1.29424893e-08])
  message: 'Optimization terminated successfully.'
     nfev: 93
      nit: 22
     njev: 31
   status: 0
  success: True
        x: array([0.99999959, 0.99999917])

SGD convergence performed a lot worse compared to the CG and BFGS methods, which only took 20 and 22 iterations respectively to find the global minumum. SGD took 1683 steps. 

#### (d) Can you draw a firm conclusion on the outcome with just one run of each method? If not, explain why ####

Yes, a firm conclusion can be drawn with just one run. Each of the three methods converges to approximately (1.0, 1.0) with one run, varying by the number of steps taken per run. Because they all converge to (1.0, 1.0) and according to the problem there is only one minimum in the Rosenbrock Banana function, we can assume the minimum is (1.0, 1.0). 

#### (e) Run all of the algorithms multiple times starting at different (x,y) positions to understand the average performance of each. Explain the relative performance of the non-stochastic and stochastic methods on the Rosenbrock Banana Function. #### 

SGD:

In [15]:
def run_multiple_times(num_runs, func, first_derivative, stepsize, 
                       starting_point=0, tol=1e-5, stochastic_injection=1):
    """Run the algorithm multiple times and return the average number of steps taken to converge. """

    steps_SGD = []
    steps_CG = []
    steps_BFGS = []

    print(f"Running {num_runs} times")

    if type(starting_point) == int:
        for i in range(num_runs):
            starting_point = np.random.uniform(-3,3,2)
            steps_SGD.append(stochastic_gradient_descent(func, 
                                                         first_derivative, 
                                                         starting_point, 
                                                         stepsize, tol, 
                                                         stochastic_injection)['steps'])
            steps_CG.append(minimize(func, starting_point, 
                                     method='CG', 
                                     options={'disp': False, 'gtol': tol}).nit)
            steps_BFGS.append(minimize(func, starting_point, 
                                       method='BFGS', 
                                       options={'disp': False, 'gtol': tol}).nit)

            # in case of error, print the starting point
            if steps_SGD[-1] < 30:
                print(f"SGD Overflow at {starting_point}")
    
    else:
        for i in range(num_runs):
            steps_SGD.append(stochastic_gradient_descent(func, 
                                                         first_derivative, 
                                                         starting_point, 
                                                         stepsize, 
                                                         tol, 
                                                         stochastic_injection)['steps'])
            steps_CG.append(minimize(func, starting_point, 
                                     method='CG', 
                                     options={'disp': False, 'gtol': tol}).nit)
            steps_BFGS.append(minimize(func, 
                                       starting_point, method='BFGS', 
                                       options={'disp': False, 'gtol': tol}).nit)
            
    print (f"Average number of steps for stochastic gradient descent: {np.mean(steps_SGD)}")
    print (f"Average number of steps for conjugate gradient descent: {np.mean(steps_CG)}")
    print (f"Average number of steps for BFGS: {np.mean(steps_BFGS)}")

In [16]:
run_multiple_times(5, rosenbrock, derive_rosenbrock, 0.01)

Running 5 times
func:'stochastic_gradient_descent' took: 0.1020 sec
func:'stochastic_gradient_descent' took: 0.2174 sec
func:'stochastic_gradient_descent' took: 0.0752 sec
func:'stochastic_gradient_descent' took: 0.0804 sec
func:'stochastic_gradient_descent' took: 0.0770 sec
Average number of steps for stochastic gradient descent: 2751.6
Average number of steps for conjugate gradient descent: 16.2
Average number of steps for BFGS: 20.2


The average number of steps for stochastic gradient descent is significantly higher than those of CG and BFGS. In addition, in the range of (-3,3), there are some starting positions that cause the SGD function to crash due to overflow of the position values.

## 4. Stochastic Gradient Descent with Momentum (SGDM) ##

The Rosenbrock Banana function with one minimum is not the best way to illustrate the power of the SGD or SGDM method. Hence we next investigate the Three-Hump Camel function:
$$𝑓(𝑥, 𝑦) = 2𝑥^2 − 1.05𝑥^4 + 𝑥^6⁄6 + 𝑥𝑦 + 𝑦^2 $$
$$𝑥 ∈ [−2,2], 𝑦 ∈ [−2,2] $$
which is a convex function with three minima. This defines our first “multiple minima” problem where there is a global solution as well as two less optimal solutions.


#### (a) Utilize SGD to find the global minimum, and compare it to CG or BFGS as you did in (2e). Starting from [-1.5,-1.5], converge function and gradient to tolerance = 1 × 10-5 with stepsize =0.1. On average, did you get a better result in finding the global minimum with SGD in terms of fewer steps on average? ####

In [17]:
def three_hump_camel(x):
    return 2*x[0]**2 - 1.05*x[0]**4 + x[0]**6/6 + x[0]*x[1] + x[1]**2

def derive_three_hump_camel(x):
    return np.array([4*x[0] - 4.2*x[0]**3 + x[0]**5 + x[1], x[0] + 2*x[1]])

In [18]:
stochastic_gradient_descent(three_hump_camel, 
                            derive_three_hump_camel, 
                            np.array([-1.5, -1.5]), 0.1)

func:'stochastic_gradient_descent' took: 0.0032 sec


{'minimum': array([ 2.23473805e-06, -2.14392181e-06]),
 'evaluation': 9.793405418418065e-12,
 'steps': 44,
 'path': array([[-1.50000000e+00, -1.50000000e+00],
        [-1.08417542e+00, -7.24427140e-01],
        [-1.14034094e+00, -1.50327565e-01],
        [-9.24169333e-01, -9.10551337e-02],
        [-4.58843570e-01,  1.66660707e-01],
        [-4.42274477e-01,  9.66614342e-02],
        [-2.78875832e-01, -1.32881109e-01],
        [ 4.32439197e-01, -8.34224026e-02],
        [ 2.06497774e-01,  7.61729246e-02],
        [-1.47721838e-02, -1.42384892e-01],
        [ 1.05530716e-01, -8.77611724e-02],
        [ 2.80383806e-02,  1.68602341e-02],
        [-1.79883376e-02,  4.15716593e-02],
        [-2.64389950e-04,  4.32803303e-02],
        [ 7.67938893e-03,  3.55718232e-02],
        [-1.08618135e-02, -4.64399986e-03],
        [-8.33474890e-03,  1.01089670e-02],
        [ 8.30623060e-04,  1.44929358e-02],
        [-5.09283779e-03,  5.05110371e-03],
        [-1.02820567e-03,  6.98359326e-03],
     

In [19]:
minimize(three_hump_camel, np.array([-1.5, -1.5]), 
         method='BFGS', options={'disp': True, 'gtol': 1e-5})

Optimization terminated successfully.
         Current function value: 0.298638
         Iterations: 8
         Function evaluations: 30
         Gradient evaluations: 10


      fun: 0.29863844223686065
 hess_inv: array([[ 0.08568628, -0.04289906],
       [-0.04289906,  0.51091221]])
      jac: array([ 1.34110451e-07, -1.49011612e-08])
  message: 'Optimization terminated successfully.'
     nfev: 30
      nit: 8
     njev: 10
   status: 0
  success: True
        x: array([-1.74755234,  0.87377616])

In [20]:
minimize(three_hump_camel, np.array([-1.5, -1.5]), method='CG', 
         options={'disp': True, 'gtol': 1e-5})

Optimization terminated successfully.
         Current function value: 0.298638
         Iterations: 7
         Function evaluations: 63
         Gradient evaluations: 21


     fun: 0.2986384422397371
     jac: array([8.40425491e-06, 7.45058060e-07])
 message: 'Optimization terminated successfully.'
    nfev: 63
     nit: 7
    njev: 21
  status: 0
 success: True
       x: array([-1.74755166,  0.87377619])

In [21]:
run_multiple_times(5, three_hump_camel, derive_three_hump_camel, 
                   0.1, np.array([-1.5, -1.5]))

Running 5 times
func:'stochastic_gradient_descent' took: 0.0071 sec
func:'stochastic_gradient_descent' took: 0.0039 sec
func:'stochastic_gradient_descent' took: 0.0021 sec
func:'stochastic_gradient_descent' took: 0.0019 sec
func:'stochastic_gradient_descent' took: 0.0044 sec
Average number of steps for stochastic gradient descent: 68.6
Average number of steps for conjugate gradient descent: 7.0
Average number of steps for BFGS: 8.0


In terms of average steps to find the minimum, CG and BFGS were better. However, the three functions all do not converge to the global minimum (0,0) a lot of the time. 

#### (b) Implement the SGDM algorithm with momentum 𝛾 = 0.9. Now use SGD with Momentum to find the global minimum. Again start from [-1.5,-1.5] with stepsize = 0.1 and converge function  and gradient to tolerance = $1×10^{-5}$. On average,did you ge a better result using SGDM compared to SGD, CG or BFGS in finding the global minimum in terms of fewer steps? #### 

In [22]:
@timeit
def SGDM(func,first_derivative,starting_point,stepsize,momentum=0.9,
         tol=1e-5,stochastic_injection=1):
    # evaluate the gradient at starting point
    
    count = 0
    visited = [starting_point]
    deriv = first_derivative(starting_point)
    previous_direction = np.zeros(len(starting_point))

    while LA.norm(deriv) > tol and count < 1e5:
        if stochastic_injection>0:
            # formulate a stochastic_deriv that is the same norm as deriv
            stochastic_deriv = np.random.normal(0,1,2) # random vector with mean 0 and std 1 
            stochastic_deriv = stochastic_deriv/LA.norm(stochastic_deriv)*LA.norm(deriv) 
        else: 
            stochastic_deriv = np.zeros(len(starting_point))
        
        direction =- (deriv + stochastic_injection*stochastic_deriv)
        direction = momentum*previous_direction + direction
        new_position = visited[-1] + stepsize*direction
        previous_direction = direction
        deriv = first_derivative(new_position)

        if func(new_position) < func(visited[-1]):
            stepsize *= 1.2
        else:
            stepsize *= 0.5

            if stepsize < 1e-5:
                previous_direction = np.zeros(len(starting_point))

        visited.append(new_position)
        count += 1

    return {"minimum":new_position,"evaluation":func(new_position), "steps":count, "path":np.asarray(visited)}


In [23]:
SGDM(three_hump_camel, derive_three_hump_camel, np.array([-1.5, -1.5]), 0.1)

func:'SGDM' took: 0.0296 sec


{'minimum': array([-9.81736841e-07,  2.06759961e-06]),
 'evaluation': 4.172743886762954e-12,
 'steps': 438,
 'path': array([[-1.50000000e+00, -1.50000000e+00],
        [-1.02558206e+00, -7.95830937e-01],
        [-1.21032131e-01, -1.48924517e-02],
        [ 9.91888393e-01,  8.85307298e-01],
        [ 1.39329757e+00,  1.32434774e+00],
        [ 1.41674285e+00,  1.30301602e+00],
        [ 1.57086765e+00,  1.17993613e+00],
        [ 1.81992517e+00,  6.75040380e-01],
        [ 1.71518685e+00, -2.09723771e-01],
        [ 1.67878562e+00, -1.26398891e+00],
        [ 1.74996931e+00, -2.44931496e+00],
        [ 1.87085348e+00, -2.68580215e+00],
        [ 1.89612227e+00, -2.63913817e+00],
        [ 1.81747401e+00, -2.53937005e+00],
        [ 1.82496475e+00, -2.40652709e+00],
        [ 1.94992709e+00, -2.09038012e+00],
        [ 2.11721214e+00, -1.60792462e+00],
        [ 1.75015471e+00, -1.33351011e+00],
        [ 1.38135730e+00, -9.88182278e-01],
        [ 1.21673372e+00, -8.35580262e-01],
    

In [24]:
steepest_descent(three_hump_camel, derive_three_hump_camel, np.array([-1.5, 1.5]), 0.1, 1e-5)

func:'steepest_descent' took: 0.0013 sec


{'minimum': array([-1.74755321,  0.87377874]),
 'evaluation': 0.2986384422457916,
 'steps': 31,
 'path': array([[-1.5       ,  1.5       ],
        [-1.708125  ,  1.35      ],
        [-1.81711465,  1.230975  ],
        [-1.72366291,  1.13811871],
        [-1.81648029,  1.04263383],
        [-1.64507648,  0.98689808],
        [-1.75463477,  0.95281643],
        [-1.75356293,  0.93402985],
        [-1.75148293,  0.91693557],
        [-1.75057007,  0.90217498],
        [-1.7487293 ,  0.89061279],
        [-1.74937157,  0.88222911],
        [-1.74511161,  0.87755848],
        [-1.75746724,  0.87384145],
        [-1.73451962,  0.87565891],
        [-1.74891576,  0.87409876],
        [-1.7470985 ,  0.87417881],
        [-1.74788916,  0.87401041],
        [-1.74726925,  0.87398928],
        [-1.74797241,  0.87385268],
        [-1.74748766,  0.87387839],
        [-1.7475903 ,  0.87384729],
        [-1.7475362 ,  0.87383283],
        [-1.74757828,  0.87381129],
        [-1.74752235,  0.8738024

In [25]:
stochastic_gradient_descent(three_hump_camel, 
                            derive_three_hump_camel, 
                            np.array([-1.5, -1.5]), 0.1)

func:'stochastic_gradient_descent' took: 0.0042 sec


{'minimum': array([-1.31612992e-06,  4.18231445e-07]),
 'evaluation': 3.0888665563219564e-12,
 'steps': 47,
 'path': array([[-1.50000000e+00, -1.50000000e+00],
        [-1.49471950e+00, -1.50104590e+00],
        [-8.53041164e-01, -8.18713498e-01],
        [-3.77820990e-01, -2.78418024e-02],
        [-3.85847412e-01,  9.04796580e-02],
        [ 9.35907594e-02,  1.23051873e-02],
        [ 2.85125131e-02,  7.79807344e-02],
        [-4.04316414e-02, -5.57376922e-02],
        [ 1.02993667e-01,  6.73605948e-02],
        [-4.09840539e-02,  9.94505511e-02],
        [-6.05078782e-02,  5.02503045e-02],
        [-4.84722890e-02,  7.37971522e-02],
        [-1.74591638e-02,  9.08256664e-02],
        [-3.08650602e-03,  8.40082170e-02],
        [ 1.35497327e-02,  3.85768405e-02],
        [-2.91972366e-02, -3.30115328e-04],
        [ 2.85315452e-02, -1.09158421e-02],
        [ 1.24649411e-02, -4.15269526e-02],
        [ 3.50068054e-03, -3.86797070e-02],
        [ 8.99219691e-03, -3.94344277e-02],
    

In [26]:
# run SGDM multiple times starting at different (x,y) positions and return average steps
def run_multiple_times_SGDM(num_runs, func, first_derivative, starting_point, stepsize):

    steps = []

    print(f"Running {num_runs} times")

    for i in range(num_runs):
        steps.append(SGDM(func, first_derivative, starting_point, stepsize)['steps'])

        # in case of error, print the starting point
        if steps[-1] < 50:
            print(f"SGD Overflow at {starting_point}")

    return np.mean(steps)

In [29]:
run_multiple_times_SGDM(5, three_hump_camel, derive_three_hump_camel, np.array([-1.5, -1.5]), 0.1)

Running 5 times
func:'SGDM' took: 0.0278 sec
func:'SGDM' took: 0.0172 sec
func:'SGDM' took: 0.0155 sec
func:'SGDM' took: 0.0191 sec
func:'SGDM' took: 0.0133 sec


418.6

In terms of number of steps, SGDM performs worse than the other algorithms. Although SGDM seems to find the global minimum more often than CG or BFGS, SGD is also able to find the global minimum and converges to very small values a good amount of time as well. In addition, SGD only takes around 60 steps to do so. 